In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm

from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier

from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from itertools import combinations

import tensorflow as tf
tf.keras.backend.clear_session()

In [14]:
dfX = pd.read_csv("dataX.csv")
dfy = pd.read_csv("datay.csv")
dfX = dfX.drop(['Unnamed: 0'], axis = 1)
dfy = dfy.drop(['Unnamed: 0'], axis = 1)

In [ ]:
X

In [ ]:
# create scaler from training data
scaler = StandardScaler().fit(X_train)

# apply same scaler to both train and test data
# this steps produces numpy matrices
X_train_t = scaler.transform(X_train)
X_test_t = scaler.transform(X_test)

# convert numpy matrices to pandas dataframes
# and assign back to X_train and X_test
X_train_t = pd.DataFrame(X_train_t, columns=X_train.columns)
X_test_t = pd.DataFrame(X_test_t, columns=X_train.columns)

# adjust categorical variables, we only want to standardize the numeric predictors and not the categorical variables
X_train_t[categorical_variables] = X_train[categorical_variables].values
X_test_t[categorical_variables] = X_test[categorical_variables].values

# overwrite X_train and X_test with the standardized values
X_train = X_train_t
X_test = X_test_t

In [15]:
dfX.head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,1.0,1.0,0.300,1.0,1.0,1.0,1.0,0.000000,9.666667,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.000000,1.0,6.0
1,1.0,1.0,0.325,1.0,1.0,1.0,1.0,0.000000,15.541667,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,4.0
2,1.0,1.0,0.300,1.0,1.0,1.0,1.0,0.333333,22.000000,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.972222,1.0,7.0
3,1.0,1.0,1.000,1.0,1.0,1.0,1.0,1.000000,0.000000,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,0.0
4,1.0,1.0,1.000,1.0,1.0,1.0,1.0,1.000000,0.000000,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,0.0


In [16]:
dfy.head()

,0
0,1
1,1
2,1
3,1
4,1


In [3]:
def accuracy(preds, actual):
    return ((preds == actual) * 1).sum() / len(actual)

In [ ]:
model2 = tf.keras.models.Sequential()
model2.add(tf.keras.layers.Dense(200, activation = 'relu',
                               kernel_initializer='random_normal',
                               bias_initializer='random_normal',
                               kernel_regularizer = tf.keras.regularizers.l1(0.003)))
model.add(tf.keras.layers.Dropout(0.1))

model2.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(lr = 0.001)

lrreduce = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.7,
    patience=30,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0,
)
es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=400,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

model2.compile(optimizer = optimizer,
              loss = loss_fn,
              metrics = ["accuracy"])

history = model2.fit(X_train_np, y_train_np, validation_split = 0.2, epochs = 400, callbacks = [es, lrreduce])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,6))

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Model accuracy')
ax[0].set_ylabel('accuracy')
ax[0].set_xlabel('epoch')
ax[0].legend(['train', 'val'], loc='best')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Model loss')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('epoch')
ax[1].legend(['train', 'val'], loc='best')

In [ ]:
accuracy(model2.predict_classes(X_test_np).flatten(), y_test_np)